### import needed packages

In [2]:
import requests as r
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd



###  making a list of column name
which doesn't need to be iterated every time <br>
list name : column_names

In [3]:
# making a fixed column names list

column_name_unformat = []
column_names = []

column_names.append('date') 

# getting the url
page_url = "http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=C0A9G0&stname=%25E5%258D%2597%25E6%25B8%25AF&datepicker=2016-02-01"
res = r.get(page_url)
res.encoding = 'utf-8' # 調整Response obect的encoding，不然中文是亂碼
soup = BeautifulSoup(res.text, 'html5lib')

for i in range(9, 24):
    col_title_unformat = soup.select("th")[i]
    col_title = "%s"%col_title_unformat
    
    column_name_unformat.append(col_title)
    #col_title = col_title.split("<th>")[1]
    #print(col_title)


for i in column_name_unformat :
    col_title = i.split('<th>')[1].split("</th>")[0].replace("<br/>", " ")
    
    column_names.append(col_title)
    
    #print(col_title)

   
column_names

['date',
 '觀測時間 (LST) ObsTime',
 '測站氣壓 (hPa) StnPres',
 '海平面氣壓 (hPa) SeaPres',
 '氣溫 (℃) Temperature',
 '露點溫度 (℃) Td dew point',
 '相對溼度 (%) RH',
 '風速 (m/s) WS',
 '風向 (360degree) WD',
 '最大陣風 (m/s) WSGust',
 '最大陣風風向 (360degree) WDGust',
 '降水量 (mm) Precp',
 '降水時數 (hr) PrecpHour ',
 '日照時數 (hr) SunShine',
 '全天空日射量 (MJ/㎡) GloblRad',
 '能見度 (km) Visb']

### making a crawler that can crawl over date pages (for a whole month)

In [ ]:
page_url = "http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=C0A9G0&stname=%25E5%258D%2597%25E6%25B8%25AF&datepicker={}"
date = []

for i in range(1, 30) :
    if i<10 :
        i = "0"+"%s"%i
    #print(i)
    
    feb_date = "2016-02-%s"%i
    
    feb_page = page_url.format(feb_date)
    #print(feb_page)
    
    dates = "2016/02/{}"
    date_each = dates.format(i)
    date.append(date_each)
    
    feb_res = r.get(feb_page)
    feb_res.encoding = "utf-8"
    soup = BeautifulSoup(feb_res.text, 'html5lib')
     
    